In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas_ta
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=53b3e56434a3ea23275684f68e80b84757be5ce611608a179dcfa189c5e4f606
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
# multivariate multi-step encoder-decoder lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM , GRU
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import math
import seaborn as sns
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from pandas.core.frame import DataFrame
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.optimizers import Adam
from keras.layers import Conv1D, Conv2D, MaxPooling2D, MaxPooling1D
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import MinMaxScaler
import csv
import collections
from scipy.stats import zscore
from datetime import datetime
import tensorflow as tf
import collections
from scipy.stats import zscore
from datetime import datetime
import pickle
from keras.layers import Embedding, Bidirectional, SimpleRNN

In [ ]:
print(keras.__version__)
print(tf.__version__)

3.5.0
2.17.1


## Required functions

### Company data feature engineering functions

In [ ]:
# fill missing values with a value at the same time one day ago
def fill_missing(values):
 one_day = 1
 for row in range(values.shape[0]):
  for col in range(values.shape[1]):
    if np.isnan(values[row, col]):
     values[row, col] = values[row - one_day, col]

def normalized_df(df):
    normalized_df=(df-df.mean())/df.std()
    return normalized_df

#def zlema(src, length):
#    lag = round((length - 1) / 2)
#    ema_data = src + (src - src[lag])
#    zl= ta.ema(ema_data,length = 5)
#    return zl


In [ ]:
def get_company_df(dataset,company_name,start_date):
  company = dataset[(dataset['trading_name '] == company_name)]
  new_date_range = pd.date_range(start=start_date, end="2020-03-05", freq="D")
  company = company.reindex(new_date_range, fill_value=np.NaN)
  company = company.drop(["symbol",'name','trading_name ','sectoer'], axis='columns')
  company.replace('?', np.NaN, inplace=True)
  company = company.astype('float32')
  fill_missing(company.values)
  values = company.values
  company['zg'],company['sg'],company['xg'],company['atr'] = ta.aberration(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['lower'],company['mid'],company['upper'] = ta.accbands(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['ad'] = ta.ad(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '],length=5)
  company['adosc'] = ta.adosc(high=company['high'],low= company['low'],volume=company['volume_traded '],close=company['close'],fast=1,slow=5)
  company['adx'],company['dmp'],company['dmn'] = ta.adx(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['alma'] = ta.alma(company['close'],length=5)
  company['amat1'], company['amat2']= ta.amat(company['close'],length=5)
  company['ao'] = ta.ao(high=company['high'],low= company['low'],close=company['close'],fast=1,slow=5)
  company['aobv1'],company['aobv2'],company['aobv3'],company['aobv4'],company['aobv5'],company['aobv6'],company['aobv7'] = ta.aobv(company['close'],company['volume_traded '])
  company['apo'] = ta.apo(company['close'],fast=1,slow=5)
  company['aroon_up'],company['aroon_down'],company['aroon_osc'] = ta.aroon(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['atr']= ta.atr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['bbands_lower'],company['bbands_mid'],company['bbands_upper'],company['bbands_bandwidth'],company['bbands_percent'] = ta.bbands(company['close'],length=5)
  company['bias'] = ta.bias(company['close'],length=5)
  company['bop']= ta.bop(open_=company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  company['ar'],company['br']= ta.brar(open_ = company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cci']= ta.cci(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cfo'] = ta.cfo(company['close'],length=5)
  company['cg'] = ta.cg(company['close'],length=5)
  company['chop']= ta.chop(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cksp_long'], company['cksp_short']= ta.cksp(high=company['high'],low= company['low'],close=company['close'])
  company['cmf']= ta.cmf(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '],open = company['open'],length=5)
  company['cmo'] = ta.cmo(company['close'],length=5)
  company['coppock'] = ta.coppock(company['close'],length=5,fast=6,slow=11)
  company['cti'] = ta.cti(company['close'],length=5)
  company['decay'] = ta.decay(company['close'],length=5)
  company['decreasing'] = ta.decreasing(company['close'],length=5)
  company['dema'] = ta.dema(company['close'],length=5)
  company['dmp'],company['dmn']= ta.dm(high=company['high'],low= company['low'],length=5)
  company['doncian_lower'],company['doncian_mid'],company['doncian_upper']= ta.donchian(high=company['high'],low= company['low'],lower_length=5,upper_length=5)
  company['dpo'] = ta.dpo(company['close'],length=5)
  company['ebsw'] = ta.ebsw(company['close'],length=5)
  company['efi'] = ta.efi(company['close'],company['volume_traded '],length=5)
  company['ema'] = ta.ema(company['close'],length=5)
  company['entropy'] = ta.entropy(company['close'],length=5)
  company['eom']= ta.eom(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '],length=5)
  company['er'] = ta.er(company['close'],length=5)
  company['eri_bull'],company['eri_bear'] = ta.eri(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['fisher1'],company['fisher2'] = ta.fisher(high=company['high'],low= company['low'],length=5)
  company['fwma'] = ta.fwma(company['close'],length=5)
  company['hilo'],company['hilol'],company['hilos']=ta.hilo(close=company['close'],high=company['high'],low= company['low'],high_length=5,low_length=5)
  company['hma'] = ta.hma(company['close'],length=5)
  company['hl2'] = ta.hl2(company['high'],company['low'])
  company['hlc3'] = ta.hlc3(company['high'],company['low'],company['close'])
  company['hwm'],company['hwu'],company['hwl'] = ta.hwc(company['close'])
  company['hwma '] = ta.dema(company['close'])
  company['increasing'] = ta.increasing(company['close'],length=5)
  company['inertia'] = ta.inertia(high=company['high'],low= company['low'],close=company['close'],open = company['open'],rvi_length=5,length=5)
  company['jma'] = ta.jma(company['close'],length=5)
  company['kama'] = ta.kama(company['close'],length=5)
  company['kcl'],company['kcb'],company['kcu']= ta.kc(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['kdj1'],company['kdj2'],company['kdj3'] = ta.kdj(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['kurtosis'] = ta.kurtosis(company['close'],length=5)
  company['kvo'],company['kvo_s'] = ta.kvo(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '])
  company['linreg'] = ta.linreg(company['close'],length=5)
  company['log_return'] = ta.log_return(company['close'],length=5)
  company['macd'],company['macd_hist'],company['macd_sig'] = ta.macd(company['close'])
  company['mad'] = ta.mad(company['close'],length=5)
  company['massi'] = ta.massi(high=company['high'],low= company['low'],fast=1,slow=5)
  company['median'] = ta.median(company['close'],length=5)
  company['mfi'] = ta.mfi(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '],length=5)
  company['midpoint'] = ta.midpoint(company['close'],length=5)
  company['midprice'] = ta.midprice(high=company['high'],low= company['low'],length=5)
  company['mom'] = ta.mom(company['close'],length=5)
  company['natr'] = ta.natr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['nvi'] = ta.nvi(company['close'],company['volume_traded '],length=5)
  company['obv'] = ta.obv(company['close'],company['volume_traded '])
  company['pdist'] = ta.pdist(open_=company['open'],high=company['high'],low= company['low'],close=company['close'])
  company['percent_return'] = ta.percent_return(company['close'],length=5)
  company['pgo'] = ta.pgo(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['psl'] = ta.psl(company['close'],company['open'],length=5)
  company['pvi'] = ta.pvi(company['close'],company['volume_traded '],length=5)
  company['pvol'] = ta.pvol(company['close'],company['volume_traded '])
  company['pvt'] = ta.pvt(company['close'],company['volume_traded '])
  company['pwma'] = ta.pwma(company['close'],length=5)
  company['qqe'],company['rsi_ma'] ,company['qqel'] ,company['qqes']  = ta.qqe(company['close'],length=5)
  company['qstick'] = ta.qstick(company['open'],company['close'],length=5)
  company['quantile'] = ta.quantile(company['close'],length=5)
  company['rma'] = ta.rma(company['close'],length=5)
  company['roc'] = ta.roc(company['close'],length=5)
  company['rsi'] = ta.rsi(company['close'],length=5)
  company['rsx'] = ta.rsx(company['close'],length=5)
  company['rvgi1'],company['rvgi2'] = ta.rvgi(open_=company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  #company['rvil'],company['rvib'],company['rviu'],company['rvi1'],company['rvi2'],company['rvi3'],company['rvi4'],company['rvi5'],company['rvi6'],company['rvi7'],company['rvi8'],company['rvi9'],company['rvi10'],company['rvi11'] = ta.rvi(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['sinwma'] = ta.sinwma(company['close'],length=5)
  company['skew'] = ta.skew(company['close'],length=5)
  company['slope'] = ta.slope(company['close'],length=5)
  company['sma'] = ta.sma(company['close'],length=5)
  company['smi'],company['smi_s'],company['smi_osc'] = ta.smi(company['close'])
  company['ssf'] = ta.ssf(company['close'],length=5)
  company['stc'],company['stc_macd'],company['stc_stoch'] = ta.stc(company['close'],tclength=5)
  company['stdev'] = ta.stdev(company['close'],length=5)
  company['stoch_k'],company['stoch_d'] = ta.momentum.stoch(high=company['high'],low= company['low'],close=company['close'])
  company['stochrsi_k'],company['stochrsi_d']= ta.stochrsi(company['close'],length=5)
  company['swma'] = ta.swma(company['close'],length=5)
  company['t3'] = ta.t3(company['close'],length=5)
  company['tema'] = ta.tema(company['close'],length=5)
  company['trima'] = ta.trima(company['close'],length=5)
  company['trix1'],company['trix2']  = ta.trix(company['close'],length=5)
  company['true_range'] = ta.true_range(high=company['high'],low= company['low'],close=company['close'])
  company['tema'] = ta.tema(company['close'],length=5)
  company['ui'] = ta.ui(high=company['high'],close =company['close'],length=5)
  company['uo'] = ta.uo(company['high'],company['low'],company['close'],fast=1,medium=3,slow=5)
  company['variance'] = ta.variance(company['close'],length=5)
  company['vhf'] = ta.vhf(company['close'],length=5)
  company['vidya'] = ta.vidya(company['close'],length=5)
  company['vortex1'],company['vortex2'] = ta.vortex(company['high'],company['low'],company['close'],length=5)
  company['vp1'],company['vp2'],company['vp3'],company['vp4'],company['vp5'],company['vp6'] = ta.vp(company['close'],company['volume_traded '])
  company['vwap'] = ta.vwap(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '])
  company['vmwa'] = ta.vwma(volume=company['volume_traded '],close=company['close'],length=5)
  company['wcp'] = ta.wcp(high=company['high'],low= company['low'],close=company['close'])
  company['willr'] = ta.willr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['wma'] = ta.wma(company['close'],length=5)
  company['zlma'] = ta.zlma(company['close'],length=5)
  company['zscore'] = ta.zscore(company['close'],length=5)
  #company['zlema'] = zlema(company['close'],length=5)
  company = company.fillna(0)
  #selected features based on corrlation sheet
  selected_ti = ['open','high','low','close','alma','decay','dema','ema','fwma','hma','hl2','hlc3','hwma ','jma','kama','linreg','median','midpoint','midprice','pwma','quantile','rma','sinwma','sma','ssf','swma','t3','tema','trima','vwap','vmwa','wcp','wma','zlma']
  new_company_df =  company[selected_ti]
  return new_company_df, selected_ti

### Company data prepration functions

In [ ]:
# convert history into inputs and outputs
def to_supervised(data, n_input, n_out):
  # flatten data
  X, y = list(), list()
  in_start = 0
	# step over the entire history one time step at a time
  for _ in range(len(data)):
    # define the end of the input sequence
    in_end = in_start + n_input
    out_end = in_end + n_out
    # ensure we have enough data for this instance
    if out_end < len(data):
      X.append(data.iloc[in_start:in_end, :])
      y.append(data.iloc[in_end:out_end, 3])
    # move along one time step
    in_start += 1
  print(f'X shape: {array(X).shape}')
  print(f'y shape: {array(y).shape}')
  return array(X), array(y)

def get_last_val_set_features(data):
  # flatten data
  X = list()
  X.append(data.iloc[len(data)-5:, :])
  return X


In [ ]:
#Test starting from 1-1-2017 based on 'Present Effictive short-term forecasts of Saudi stock price trends using technical indicators and large-scale multivariate time series'
# split a multivariate dataset into train/test sets
def prepare_company_for_training_and_testing(company,selected_ti):
  train,val, test =  company.loc[:'2015-12-31'],company.loc['1-1-2016':'31-12-2016'],company.loc['1-1-2017':]
  print(f'Train shape: {train.shape}, Val shape: {val.shape}, Test shape: {test.shape}')
  scaler = StandardScaler()
  scaler.fit(train)
  data = scaler.transform(train)
  train_normlized = DataFrame(data = data , columns = selected_ti ,index=train.index)
  scaler = StandardScaler()
  scaler.fit(val)
  data = scaler.transform(val)
  val_normlized = DataFrame(data = data , columns = selected_ti ,index=val.index)
  scaler = StandardScaler()
  scaler.fit(test)
  data = scaler.transform(test)
  test_normlized = DataFrame(data = data , columns =selected_ti,index=test.index)
  print("*******************************************")
  print("Train data after multistep approach")
  train_x_normlized, train_y_normlized = to_supervised(train_normlized, 5,5)
  print("Validation multistep approach")
  val_x_normlized,val_y_normlized = to_supervised(val_normlized, 5,5)
  print("Test multistep approach")
  last_val = get_last_val_set_features(val_normlized)
  test_x_normlized,test_y_normlized = to_supervised(test_normlized, 5,5)
  return train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized, test

### Company models creating and evaluation functions

In [ ]:
def true_y_to_evaluation_form(y_true, n_input, n_out):
  y = list()
  in_start = 0
	# step over the entire history one time step at a time
  for _ in range(len(y_true)):
    # define the end of the input sequence
    in_end = in_start + n_input
    out_end = in_end + n_out
    # ensure we have enough data for this instance
    if out_end < len(y_true):
      y.append(y_true[in_end:out_end])
    # move along one time step
    in_start += 1
  return array(y)

# make a forecast
def forecast(model, history):
	yhat = model.predict(history, verbose=0)
	# we only want the vector forecast
	return yhat

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
  scaler = StandardScaler()
  scaler.fit(actual)
  predicted_not_normlized = scaler.inverse_transform(predicted)
  mse_score= mse(actual,predicted_not_normlized)
  rmse_score = sqrt(mse_score)
  mae_score = mae(actual,predicted_not_normlized)
  final_r2_score = r2_score(actual,predicted_not_normlized,multioutput='variance_weighted')
  return mse_score, rmse_score,mae_score,final_r2_score,predicted_not_normlized

# summarize scores
def summarize_scores(name, mse_score, rmse_score,mae_score,final_r2_score):
  #s_scores = ', '.join(['%.1f' % s for s in scores])
  #print('%s: [%.3f] %s' % (name, score, s_scores))
  print(f'*************{name}**********')
  print("MAE: ",mae_score)
  print("MSE: ",mse_score)
  print("RMSE: ",rmse_score)
  print("R2: ",final_r2_score)

# evaluate a single model
def evaluate_model(test_x,test_y,model):
	# walk-forward validation over each week
  predictions = list()
  predictions = forecast(model, test_x)
  # evaluate predictions days for each week
  predictions = array(predictions)
  mse_score, rmse_score,mae_score,final_r2_score, y_hat_not_normlized = evaluate_forecasts(test_y, predictions)
  return mse_score, rmse_score,mae_score,final_r2_score,y_hat_not_normlized

In [ ]:
def get_brnn_models_info(train_x,train_y,val_x,val_y,test_x,y_test,test_y_for_evaluation,company_name,evaluation_df,predictions_df):
  # define parameters
  verbose, epochs, batch_size = 0, 32, math.floor(len(train_x)*0.05)
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
  # reshape output into [samples, timesteps, features]
  train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

  i = 2
  while i < 1030:
    # define model
    model_BRNN_prposed = Sequential()
    model_BRNN_prposed.add(Bidirectional(SimpleRNN(i, activation='relu', input_shape=(n_timesteps, n_features))))
    model_BRNN_prposed.add(Dense(5))
    model_BRNN_prposed.compile(loss='mse', optimizer='adam')
    # fit network
    model_BRNN_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p,predictions_L_p = evaluate_model(test_x,test_y_for_evaluation,model_BRNN_prposed)
    evaluation_df.loc[i,"MAE_b"],evaluation_df.loc[i,"MSE_b"],evaluation_df.loc[i,"RMSE_b"],evaluation_df.loc[i,"R2_b"] = mae_score_L_p,mse_score_L_p, rmse_score_L_p,final_r2_score_L_p
    print(f"Parameter of BRNN_ is {i}")
    pickle.dump(model_BRNN_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_BRNN_{i}' , 'wb'))
    summarize_scores('Prposed BRNN_', mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p)
    predictions_df[f"BRNN_{i}"] = predictions_L_p.flatten()
    i = i * 2

  i = 3
  while i < 1030:
    # define model
    model_BRNN_prposed = Sequential()
    model_BRNN_prposed.add(Bidirectional(SimpleRNN(i, activation='relu', input_shape=(n_timesteps, n_features))))
    model_BRNN_prposed.add(Dense(5))
    model_BRNN_prposed.compile(loss='mse', optimizer='adam')
    # fit network
    model_BRNN_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p,predictions_L_p = evaluate_model(test_x,test_y_for_evaluation,model_BRNN_prposed)
    evaluation_df.loc[i,"MAE_b"],evaluation_df.loc[i,"MSE_b"],evaluation_df.loc[i,"RMSE_b"],evaluation_df.loc[i,"R2_b"] = mae_score_L_p,mse_score_L_p, rmse_score_L_p,final_r2_score_L_p
    print(f"Parameter of BRNN_ is {i}")
    pickle.dump(model_BRNN_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_BRNN_{i}' , 'wb'))
    summarize_scores('Prposed BRNN_', mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p)
    predictions_df[f"BRNN_{i}"] = predictions_L_p.flatten()
    i = i * 3

  i = 100
  while i < 1030:
    # define model
    model_BRNN_prposed = Sequential()
    model_BRNN_prposed.add(Bidirectional(SimpleRNN(i, activation='relu', input_shape=(n_timesteps, n_features))))
    model_BRNN_prposed.add(Dense(5))
    model_BRNN_prposed.compile(loss='mse', optimizer='adam')
    # fit network
    model_BRNN_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p,predictions_L_p = evaluate_model(test_x,test_y_for_evaluation,model_BRNN_prposed)
    evaluation_df.loc[i,"MAE_l"],evaluation_df.loc[i,"MSE_b"],evaluation_df.loc[i,"RMSE_b"],evaluation_df.loc[i,"R2_b"] = mae_score_L_p,mse_score_L_p, rmse_score_L_p,final_r2_score_L_p
    print(f"Parameter of BRNN_ is {i}")
    pickle.dump(model_BRNN_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_BRNN_{i}' , 'wb'))
    summarize_scores('Prposed BRNN_', mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p)
    predictions_df[f"BRNN_{i}"] = predictions_L_p.flatten()
    i = i + 100

  return evaluation_df,predictions_df

def get_gru_models_info(train_x,train_y,val_x,val_y,test_x,y_test,test_y_for_evaluation,company_name,evaluation_df,predictions_df):
  # define parameters
  verbose, epochs, batch_size = 0, 32, math.floor(len(train_x)*0.05)
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
  # reshape output into [samples, timesteps, features]
  train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

  i = 2
  while i < 1030:
    # define model
    model_GRU_prposed = Sequential()
    model_GRU_prposed.add(GRU(i, activation='relu', input_shape=(n_timesteps, n_features)))
    model_GRU_prposed.add(Dense(5))
    model_GRU_prposed.compile(loss='mse', optimizer='adam')
    tf.keras.utils.plot_model(model_GRU_prposed)
    # fit network
    model_GRU_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_G_p, rmse_score_G_p,mae_score_G_p,final_r2_score_G_p,predictions_G_p = evaluate_model(test_x,test_y_for_evaluation,model_GRU_prposed)
    evaluation_df.loc[i,"MAE_g"],evaluation_df.loc[i,"MSE_g"],evaluation_df.loc[i,"RMSE_g"],evaluation_df.loc[i,"R2_g"] = mae_score_G_p,mse_score_G_p, rmse_score_G_p,final_r2_score_G_p
    print(f"Parameter of GRU is {i}")
    pickle.dump(model_GRU_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_GRU_{i}' , 'wb'))
    summarize_scores('Prposed GRU', mse_score_G_p, rmse_score_G_p,mae_score_G_p,final_r2_score_G_p)
    predictions_df[f"GRU{i}"] = predictions_G_p.flatten()
    i = i * 2

  i = 3
  while i < 1030:
    # define model
    model_GRU_prposed = Sequential()
    model_GRU_prposed.add(GRU(i, activation='relu', input_shape=(n_timesteps, n_features)))
    model_GRU_prposed.add(Dense(5))
    model_GRU_prposed.compile(loss='mse', optimizer='adam')
    tf.keras.utils.plot_model(model_GRU_prposed)
    # fit network
    model_GRU_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_G_p, rmse_score_G_p,mae_score_G_p,final_r2_score_G_p,predictions_G_p = evaluate_model(test_x,test_y_for_evaluation,model_GRU_prposed)
    evaluation_df.loc[i,"MAE_g"],evaluation_df.loc[i,"MSE_g"],evaluation_df.loc[i,"RMSE_g"],evaluation_df.loc[i,"R2_g"] = mae_score_G_p,mse_score_G_p, rmse_score_G_p,final_r2_score_G_p
    print(f"Parameter of GRU is {i}")
    pickle.dump(model_GRU_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_GRU_{i}' , 'wb'))
    summarize_scores('Prposed GRU', mse_score_G_p, rmse_score_G_p,mae_score_G_p,final_r2_score_G_p)
    predictions_df[f"GRU{i}"] = predictions_G_p.flatten()
    i = i * 3
  i = 100
  while i < 1030:
    # define model
    model_GRU_prposed = Sequential()
    model_GRU_prposed.add(GRU(i, activation='relu', input_shape=(n_timesteps, n_features)))
    model_GRU_prposed.add(Dense(5))
    model_GRU_prposed.compile(loss='mse', optimizer='adam')
    tf.keras.utils.plot_model(model_GRU_prposed)
    # fit network
    model_GRU_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_G_p, rmse_score_G_p,mae_score_G_p,final_r2_score_G_p,predictions_G_p = evaluate_model(test_x,test_y_for_evaluation,model_GRU_prposed)
    evaluation_df.loc[i,"MAE_g"],evaluation_df.loc[i,"MSE_g"],evaluation_df.loc[i,"RMSE_g"],evaluation_df.loc[i,"R2_g"] = mae_score_G_p,mse_score_G_p, rmse_score_G_p,final_r2_score_G_p
    print(f"Parameter of GRU is {i}")
    pickle.dump(model_GRU_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_GRU_{i}' , 'wb'))
    summarize_scores('Prposed GRU', mse_score_G_p, rmse_score_G_p,mae_score_G_p,final_r2_score_G_p)
    predictions_df[f"GRU{i}"] = predictions_G_p.flatten()
    i = i + 100
  return evaluation_df,predictions_df

def get_lstm_models_info(train_x,train_y,val_x,val_y,test_x,y_test,test_y_for_evaluation,company_name,evaluation_df,predictions_df):
  # define parameters
  verbose, epochs, batch_size = 0, 32, math.floor(len(train_x)*0.05)
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
  # reshape output into [samples, timesteps, features]
  train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

  i = 2
  while i < 1030:
    # define model
    model_LSTM_prposed = Sequential()
    model_LSTM_prposed.add(LSTM(i, activation='relu', input_shape=(n_timesteps, n_features)))
    model_LSTM_prposed.add(Dense(5))
    model_LSTM_prposed.compile(loss='mse', optimizer='adam')
    # fit network
    model_LSTM_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p,predictions_L_p = evaluate_model(test_x,test_y_for_evaluation,model_LSTM_prposed)
    evaluation_df.loc[i,"MAE_l"],evaluation_df.loc[i,"MSE_l"],evaluation_df.loc[i,"RMSE_l"],evaluation_df.loc[i,"R2_l"] = mae_score_L_p,mse_score_L_p, rmse_score_L_p,final_r2_score_L_p
    print(f"Parameter of LSTM is {i}")
    pickle.dump(model_LSTM_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_LSTM_{i}' , 'wb'))
    summarize_scores('Prposed LSTM', mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p)
    predictions_df[f"LSTM_{i}"] = predictions_L_p.flatten()
    i = i * 2

  i = 3
  while i < 1030:
    # define model
    model_LSTM_prposed = Sequential()
    model_LSTM_prposed.add(LSTM(i, activation='relu', input_shape=(n_timesteps, n_features)))
    model_LSTM_prposed.add(Dense(5))
    model_LSTM_prposed.compile(loss='mse', optimizer='adam')
    # fit network
    model_LSTM_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p,predictions_L_p = evaluate_model(test_x,test_y_for_evaluation,model_LSTM_prposed)
    evaluation_df.loc[i,"MAE_l"],evaluation_df.loc[i,"MSE_l"],evaluation_df.loc[i,"RMSE_l"],evaluation_df.loc[i,"R2_l"] = mae_score_L_p,mse_score_L_p, rmse_score_L_p,final_r2_score_L_p
    print(f"Parameter of LSTM is {i}")
    pickle.dump(model_LSTM_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_LSTM_{i}' , 'wb'))
    summarize_scores('Prposed LSTM', mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p)
    predictions_df[f"LSTM_{i}"] = predictions_L_p.flatten()
    i = i * 3

  i = 100
  while i < 1030:
    # define model
    model_LSTM_prposed = Sequential()
    model_LSTM_prposed.add(LSTM(i, activation='relu', input_shape=(n_timesteps, n_features)))
    model_LSTM_prposed.add(Dense(5))
    model_LSTM_prposed.compile(loss='mse', optimizer='adam')
    # fit network
    model_LSTM_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p,predictions_L_p = evaluate_model(test_x,test_y_for_evaluation,model_LSTM_prposed)
    evaluation_df.loc[i,"MAE_l"],evaluation_df.loc[i,"MSE_l"],evaluation_df.loc[i,"RMSE_l"],evaluation_df.loc[i,"R2_l"] = mae_score_L_p,mse_score_L_p, rmse_score_L_p,final_r2_score_L_p
    print(f"Parameter of LSTM is {i}")
    pickle.dump(model_LSTM_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_LSTM_{i}' , 'wb'))
    summarize_scores('Prposed LSTM', mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p)
    predictions_df[f"LSTM_{i}"] = predictions_L_p.flatten()
    i = i + 100

  return evaluation_df,predictions_df

In [ ]:
def train_evalute_models_for_company(train_x,train_y,val_x,val_y,test_x,y_test,test,company_name):
  eval_cols = ['MAE_c','MSE_c','RMSE_c','R2_c','MAE_g','MSE_g','RMSE_g','R2_g','MAE_l','MSE_l','RMSE_l','R2_l']
  unit_indices = [2,4,8,16,32,64,128,256,512,1024,3,9,27,81,243,729,100,200,300,400,500,600,700,800,900,1000]
  test_y_for_evaluation = true_y_to_evaluation_form(test['close'].values, 5, 5)
  evaluation_df = pd.DataFrame(columns = eval_cols, index = unit_indices)
  predictions_df = DataFrame(data = test_y_for_evaluation.flatten())
  #LSTM
  evaluation_df,predictions_df = get_lstm_models_info(train_x,train_y,val_x,val_y,test_x,y_test,test_y_for_evaluation,company_name,evaluation_df,predictions_df)
  #CNN
  evaluation_df,predictions_df = get_brnn_models_info(train_x,train_y,val_x,val_y,test_x,y_test,test_y_for_evaluation,company_name,evaluation_df,predictions_df)
  #GRU
  evaluation_df,predictions_df = get_gru_models_info(train_x,train_y,val_x,val_y,test_x,y_test,test_y_for_evaluation,company_name,evaluation_df,predictions_df)
  return evaluation_df,predictions_df



## Excution

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/KFUPM/Graduation/data/Tadawul_stcks.csv',index_col='date',parse_dates=True)
dataset.sort_values(by='date', inplace = True)

In [ ]:
selected_companies_dic = {"ADC":["ADC","2001-12-31"], "ALRAJHI":["ALRAJHI","2001-12-31"], "GACO":["GACO","2002-01-01"], "EMAAR EC":["EMAAR EC","2006-10-06"], "FITAIHI GROUP":["FITAIHI GROUP","2002-01-26"], "SARCO":["SARCO","2002-01-23"], "SAUDI ELECTRICITY":["SAUDI ELECTRICITY","2002-06-08"], "SPIMACO":["SPIMACO","2001-12-31"], "STC":["STC","2003-01-25"], "TASNEE":["TASNEE","2001-12-31"] }

### ADC

In [ ]:
adc, adc_selected_ti = get_company_df(dataset,selected_companies_dic["ADC"][0],selected_companies_dic["ADC"][1])

In [ ]:
train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(adc,adc_selected_ti)

Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 34)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)


In [ ]:
evaluation_df,predictions_df = train_evalute_models_for_company(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,selected_companies_dic["ADC"][0])

Parameter of LSTM is 2


*************Prposed LSTM**********
MAE:  0.42822972
MSE:  0.5747499
RMSE:  0.7581226068318986
R2:  0.7297321232209257


Parameter of LSTM is 4
*************Prposed LSTM**********
MAE:  0.22820178
MSE:  0.098459855
RMSE:  0.31378313326567286
R2:  0.9537006340113102


Parameter of LSTM is 8
*************Prposed LSTM**********
MAE:  0.18817404
MSE:  0.07930903
RMSE:  0.28161859201285333
R2:  0.9627060330756224


Parameter of LSTM is 16
*************Prposed LSTM**********
MAE:  0.18012708
MSE:  0.072782494
RMSE:  0.269782308774594
R2:  0.9657750399830111


...

*************Prposed GRU**********
MAE:  0.170784
MSE:  0.07118253
RMSE:  0.26680053719594454
R2:  0.9665274173512041
Parameter of GRU is 600


*************Prposed GRU**********
MAE:  0.16728908
MSE:  0.06735971
RMSE:  0.2595374891023818
R2:  0.968325031611751
Parameter of GRU is 700


*************Prposed GRU**********
MAE:  0.16461128
MSE:  0.06594844
RMSE:  0.2568042866165622
R2:  0.9689886757296601
Parameter of GRU is 800


*************Prposed GRU**********
MAE:  0.16710481
MSE:  0.06559844
RMSE:  0.2561219302412824
R2:  0.9691532515593305
Parameter of GRU is 900


*************Prposed GRU**********
MAE:  0.16343151
MSE:  0.06521513
RMSE:  0.2553725379334089
R2:  0.9693335097199814
Parameter of GRU is 1000
*************Prposed GRU**********
MAE:  0.16879137
MSE:  0.0684918
RMSE:  0.26170938389658427
R2:  0.9677926826994377


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['ADC'][0]}_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['ADC'][0]}_predictions.csv")

### ALRAJHI

In [ ]:
rajhi, rajhi_selected_ti = get_company_df(dataset,selected_companies_dic["ALRAJHI"][0],selected_companies_dic["ALRAJHI"][1])

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-5-81f741971fba>:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  company['natr'] = ta.nat

In [ ]:
train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(rajhi, rajhi_selected_ti)

Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 34)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)


In [ ]:
evaluation_df,predictions_df = train_evalute_models_for_company(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,selected_companies_dic["ALRAJHI"][0])

Parameter of LSTM is 2


*************Prposed LSTM**********
MAE:  9.149695
MSE:  110.77476
RMSE:  10.524958783066749
R2:  -0.0004820394989975074


Parameter of LSTM is 4
*************Prposed LSTM**********
MAE:  1.2010026
MSE:  2.526379
RMSE:  1.5894587470044497
R2:  0.9771825498164006


Parameter of LSTM is 8
*************Prposed LSTM**********
MAE:  1.1492224
MSE:  2.5848687
RMSE:  1.607752676722974
R2:  0.9766542987768522


Parameter of LSTM is 16
*************Prposed LSTM**********
MAE:  0.8311494
MSE:  1.3827776
RMSE:  1.1759156311905041
R2:  0.9875111951469828


...

*************Prposed GRU**********
MAE:  0.8009451
MSE:  1.3023573
RMSE:  1.141208708351435
R2:  0.9882375275076325
Parameter of GRU is 600


*************Prposed GRU**********
MAE:  0.766466
MSE:  1.2206498
RMSE:  1.1048302305999644
R2:  0.9889754791368127
Parameter of GRU is 700


*************Prposed GRU**********
MAE:  0.8558203
MSE:  1.5371319
RMSE:  1.2398112378728163
R2:  0.9861171197891683
Parameter of GRU is 800


*************Prposed GRU**********
MAE:  0.74107486
MSE:  1.1493878
RMSE:  1.0720950687585027
R2:  0.9896190989460397
Parameter of GRU is 900


*************Prposed GRU**********
MAE:  0.857856
MSE:  1.4868463
RMSE:  1.219363082835329
R2:  0.9865712840118769
Parameter of GRU is 1000
*************Prposed GRU**********
MAE:  0.77795994
MSE:  1.2264057
RMSE:  1.107432047479321
R2:  0.9889234954646787


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['ALRAJHI'][0]}_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['ALRAJHI'][0]}_predictions.csv")

### GACO

In [ ]:
gaco, gaco_selected_ti = get_company_df(dataset,selected_companies_dic["GACO"][0],selected_companies_dic["GACO"][1])

In [ ]:
train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(gaco, gaco_selected_ti)

Train shape: (5113, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5103, 5, 34)
y shape: (5103, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)


In [ ]:
evaluation_df,predictions_df = train_evalute_models_for_company(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,selected_companies_dic["GACO"][0])

Parameter of LSTM is 2


*************Prposed LSTM**********
MAE:  0.48490676
MSE:  0.57252014
RMSE:  0.756650604197995
R2:  0.6738430886298528


Parameter of LSTM is 4
*************Prposed LSTM**********
MAE:  0.23947509
MSE:  0.13235787
RMSE:  0.36381020542581777
R2:  0.924597453376984


Parameter of LSTM is 8
*************Prposed LSTM**********
MAE:  0.19801044
MSE:  0.083173916
RMSE:  0.2883988830488562
R2:  0.9526169085042037


Parameter of LSTM is 16
*************Prposed LSTM**********
MAE:  0.20465879
MSE:  0.08556756
RMSE:  0.2925193264070272
R2:  0.9512532913222171


...

*************Prposed GRU**********
MAE:  0.1813148
MSE:  0.0732557
RMSE:  0.2706579074278134
R2:  0.9582671783933895
Parameter of GRU is 600


*************Prposed GRU**********
MAE:  0.1767784
MSE:  0.07119096
RMSE:  0.26681634265029625
R2:  0.9594434330528371
Parameter of GRU is 700


*************Prposed GRU**********
MAE:  0.17451613
MSE:  0.071027875
RMSE:  0.266510553161773
R2:  0.9595363405232309
Parameter of GRU is 800


*************Prposed GRU**********
MAE:  0.17502612
MSE:  0.07147455
RMSE:  0.26734725013461613
R2:  0.9592818715753046
Parameter of GRU is 900


*************Prposed GRU**********
MAE:  0.1850908
MSE:  0.07935648
RMSE:  0.28170281621691884
R2:  0.9547916779002915
Parameter of GRU is 1000
*************Prposed GRU**********
MAE:  0.18590784
MSE:  0.076363854
RMSE:  0.27634010586637786
R2:  0.9564965153677866


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['GACO'][0]}_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['GACO'][0]}_predictions.csv")

### EMAAR EC

In [ ]:
emaar, emaar_selected_ti = get_company_df(dataset,selected_companies_dic["EMAAR EC"][0],selected_companies_dic["EMAAR EC"][1])

In [ ]:
train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(emaar, emaar_selected_ti)

Train shape: (3374, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (3364, 5, 34)
y shape: (3364, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)


In [ ]:
evaluation_df,predictions_df = train_evalute_models_for_company(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,selected_companies_dic["EMAAR EC"][0])

Parameter of LSTM is 2
*************Prposed LSTM**********
MAE:  2.0830731
MSE:  6.436319
RMSE:  2.5369901210606103
R2:  0.23813069277166815


Parameter of LSTM is 4
*************Prposed LSTM**********
MAE:  0.38415846
MSE:  0.5840628
RMSE:  0.7642400242806735
R2:  0.9308643427228489


Parameter of LSTM is 8
*************Prposed LSTM**********
MAE:  0.25192374
MSE:  0.13665764
RMSE:  0.3696723418622822
R2:  0.983823804303733


Parameter of LSTM is 16
*************Prposed LSTM**********
MAE:  0.25651535
MSE:  0.13664667
RMSE:  0.36965750781406476
R2:  0.9838251006699289


Parameter of LSTM is 32
*************Prposed LSTM**********
MAE:  0.2335275
MSE:  0.12102638
RMSE:  0.34788846179521765
R2:  0.9856740735211478


...

Parameter of GRU is 500
*************Prposed GRU**********
MAE:  0.22077782
MSE:  0.11184281
RMSE:  0.3344290823293266
R2:  0.9867611388624268


Parameter of GRU is 600
*************Prposed GRU**********
MAE:  0.21708325
MSE:  0.107953034
RMSE:  0.3285620707588374
R2:  0.9872215688554575


Parameter of GRU is 700
*************Prposed GRU**********
MAE:  0.24620792
MSE:  0.12575239
RMSE:  0.3546158335875596
R2:  0.9851146619152837


Parameter of GRU is 800
*************Prposed GRU**********
MAE:  0.22197564
MSE:  0.110202655
RMSE:  0.331967852654726
R2:  0.9869552845468738
Parameter of GRU is 900


*************Prposed GRU**********
MAE:  0.22592323
MSE:  0.11328523
RMSE:  0.33657871086870417
R2:  0.9865904033048205
Parameter of GRU is 1000
*************Prposed GRU**********
MAE:  0.2276793
MSE:  0.11531212
RMSE:  0.3395763858228792
R2:  0.98635047880679


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['EMAAR EC'][0]}_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['EMAAR EC'][0]}_predictions.csv")

### FITAIHI GROUP

In [ ]:
fg, fg_selected_ti = get_company_df(dataset,selected_companies_dic["FITAIHI GROUP"][0],selected_companies_dic["FITAIHI GROUP"][1])

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-4-81f741971fba>:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  company['natr'] = ta.nat

In [ ]:
train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(fg, fg_selected_ti)

Train shape: (5088, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5078, 5, 34)
y shape: (5078, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)


In [ ]:
evaluation_df,predictions_df = train_evalute_models_for_company(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,selected_companies_dic["FITAIHI GROUP"][0])

Parameter of LSTM is 2
*************Prposed LSTM**********
MAE:  0.29342327
MSE:  0.16489999
RMSE:  0.40607879772598054
R2:  0.7696984965878744


Parameter of LSTM is 4
*************Prposed LSTM**********
MAE:  0.3099526
MSE:  0.17068237
RMSE:  0.41313722981920814
R2:  0.7616226608893127


Parameter of LSTM is 8
*************Prposed LSTM**********
MAE:  0.1578581
MSE:  0.048719883
RMSE:  0.22072580946751083
R2:  0.9319571868628828


Parameter of LSTM is 16
*************Prposed LSTM**********
MAE:  0.1437343
MSE:  0.04341162
RMSE:  0.20835455349298646
R2:  0.9393707660790324


Parameter of LSTM is 32
*************Prposed LSTM**********
MAE:  0.13387744
MSE:  0.038713343
RMSE:  0.196757067068724
R2:  0.9459324581471739


...

Parameter of GRU is 500
*************Prposed GRU**********
MAE:  0.12808208
MSE:  0.03761876
RMSE:  0.19395556197114014
R2:  0.947461142304442


Parameter of GRU is 600
*************Prposed GRU**********
MAE:  0.12688278
MSE:  0.036896877
RMSE:  0.1920855980827171
R2:  0.9484693285674698


Parameter of GRU is 700
*************Prposed GRU**********
MAE:  0.12758026
MSE:  0.036347423
RMSE:  0.19065000065251533
R2:  0.9492367207198531


Parameter of GRU is 800
*************Prposed GRU**********
MAE:  0.12757204
MSE:  0.036259048
RMSE:  0.19041808655443862
R2:  0.9493601560722394
Parameter of GRU is 900
*************Prposed GRU**********
MAE:  0.12941504
MSE:  0.03661873
RMSE:  0.19136020534756037
R2:  0.9488578149177437


Parameter of GRU is 1000
*************Prposed GRU**********
MAE:  0.13155761
MSE:  0.03785561
RMSE:  0.19456518252613106
R2:  0.9471303419763286


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['FITAIHI GROUP'][0]}_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['FITAIHI GROUP'][0]}_predictions.csv")

### SARCO

In [ ]:
sar, sar_selected_ti = get_company_df(dataset,selected_companies_dic["SARCO"][0],selected_companies_dic["SARCO"][1])

In [ ]:
train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(sar, sar_selected_ti)

Train shape: (5091, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5081, 5, 34)
y shape: (5081, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)


In [ ]:
evaluation_df,predictions_df = train_evalute_models_for_company(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,selected_companies_dic["SARCO"][0])

Parameter of LSTM is 2


*************Prposed LSTM**********
MAE:  1.6516554
MSE:  5.5242586
RMSE:  2.35037414331983
R2:  0.7748661446321802
Parameter of LSTM is 4


*************Prposed LSTM**********
MAE:  1.1475196
MSE:  2.5758445
RMSE:  1.60494377667596
R2:  0.8950249033620232
Parameter of LSTM is 8


*************Prposed LSTM**********
MAE:  0.92120254
MSE:  2.1717205
RMSE:  1.473675847926111
R2:  0.9114944040488966
Parameter of LSTM is 16


*************Prposed LSTM**********
MAE:  0.9557821
MSE:  2.225078
RMSE:  1.4916695699539204
R2:  0.909319891912868
Parameter of LSTM is 32


...

*************Prposed GRU**********
MAE:  0.8405536
MSE:  1.7336594
RMSE:  1.3166849990164036
R2:  0.9293470318894999
Parameter of GRU is 600


*************Prposed GRU**********
MAE:  0.89632
MSE:  2.1473174
RMSE:  1.4653727885815886
R2:  0.9124889428990133
Parameter of GRU is 700


*************Prposed GRU**********
MAE:  0.78809774
MSE:  1.6828121
RMSE:  1.2972324751903244
R2:  0.9314192412527107
Parameter of GRU is 800


*************Prposed GRU**********
MAE:  0.77505034
MSE:  1.5642135
RMSE:  1.2506852179217611
R2:  0.9362525430720108
Parameter of GRU is 900


*************Prposed GRU**********
MAE:  0.78152287
MSE:  1.5634587
RMSE:  1.2503834136402192
R2:  0.9362833227629549
Parameter of GRU is 1000
*************Prposed GRU**********
MAE:  0.78861505
MSE:  1.5625461
RMSE:  1.2500184534618104
R2:  0.9363205144360867


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['SARCO'][0]}_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['SARCO'][0]}_predictions.csv")

### SAUDI ELECTRICITY

In [ ]:
se, se_selected_ti = get_company_df(dataset,selected_companies_dic["SAUDI ELECTRICITY"][0],selected_companies_dic["SAUDI ELECTRICITY"][1])

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
<ipython-input-5-81f741971fba>:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  company['natr'] = ta.natr(high=company['high'],low= company['low'],close=company['close'],length=5)
<ipython-input-5-81f741971fba>:75: PerformanceWarning: DataFrame is highl

In [ ]:
train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(se, se_selected_ti)

Train shape: (4955, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (4945, 5, 34)
y shape: (4945, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)


In [ ]:
evaluation_df,predictions_df = train_evalute_models_for_company(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,selected_companies_dic["SAUDI ELECTRICITY"][0])

Parameter of LSTM is 2


*************Prposed LSTM**********
MAE:  0.8775693
MSE:  1.3860323
RMSE:  1.17729874836881
R2:  0.8407478944137543
Parameter of LSTM is 4


*************Prposed LSTM**********
MAE:  0.41038713
MSE:  0.3258384
RMSE:  0.5708225529991625
R2:  0.9625618856988247
Parameter of LSTM is 8


*************Prposed LSTM**********
MAE:  0.43526077
MSE:  0.3493917
RMSE:  0.5910936464193134
R2:  0.9598556436811319
Parameter of LSTM is 16


*************Prposed LSTM**********
MAE:  0.3740389
MSE:  0.2873968
RMSE:  0.536094010969258
R2:  0.9669787216373922
Parameter of LSTM is 32


...

*************Prposed GRU**********
MAE:  0.32645458
MSE:  0.2315313
RMSE:  0.48117700184157775
R2:  0.9733975340022641
Parameter of GRU is 600


*************Prposed GRU**********
MAE:  0.33320123
MSE:  0.24311905
RMSE:  0.49307103554359527
R2:  0.9720661477383522
Parameter of GRU is 700


*************Prposed GRU**********
MAE:  0.3133695
MSE:  0.22457698
RMSE:  0.47389553688082414
R2:  0.9741965780797358
Parameter of GRU is 800


*************Prposed GRU**********
MAE:  0.32183063
MSE:  0.22762041
RMSE:  0.4770958058283024
R2:  0.9738468968669994
Parameter of GRU is 900


*************Prposed GRU**********
MAE:  0.35136423
MSE:  0.2402087
RMSE:  0.49011090612152575
R2:  0.972400525812312
Parameter of GRU is 1000
*************Prposed GRU**********
MAE:  0.31594452
MSE:  0.22600384
RMSE:  0.4753986122883421
R2:  0.974032637161532


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['SAUDI ELECTRICITY'][0]}_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['SAUDI ELECTRICITY'][0]}_predictions.csv")

### SPIMACO

In [ ]:
sp, sp_selected_ti = get_company_df(dataset,selected_companies_dic["SPIMACO"][0],selected_companies_dic["SPIMACO"][1])

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
<ipython-input-8-81f741971fba>:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  company['natr'] = ta.nat

In [ ]:
train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(sp, sp_selected_ti)

Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 34)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)


In [ ]:
evaluation_df,predictions_df = train_evalute_models_for_company(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,selected_companies_dic["SPIMACO"][0])

Parameter of LSTM is 2


*************Prposed LSTM**********
MAE:  0.9008317
MSE:  1.5561559
RMSE:  1.2474597869385156
R2:  0.872275624294527


Parameter of LSTM is 4
*************Prposed LSTM**********
MAE:  1.0245922
MSE:  2.0840685
RMSE:  1.443630332446095
R2:  0.8289462585726065


Parameter of LSTM is 8
*************Prposed LSTM**********
MAE:  0.43790603
MSE:  0.36860174
RMSE:  0.6071258019608339
R2:  0.9697463258237149


Parameter of LSTM is 16
*************Prposed LSTM**********
MAE:  0.4020514
MSE:  0.32217455
RMSE:  0.5676042187147513
R2:  0.9735569262063408


...

*************Prposed GRU**********
MAE:  0.35497576
MSE:  0.27612168
RMSE:  0.5254728118258548
R2:  0.9773368073551607
Parameter of GRU is 600


*************Prposed GRU**********
MAE:  0.35178688
MSE:  0.27838618
RMSE:  0.5276231378858179
R2:  0.9771509401560077
Parameter of GRU is 700


*************Prposed GRU**********
MAE:  0.35415038
MSE:  0.27478027
RMSE:  0.5241948811630079
R2:  0.9774469087334365
Parameter of GRU is 800


*************Prposed GRU**********
MAE:  0.35449392
MSE:  0.27565783
RMSE:  0.5250312682331673
R2:  0.9773748584214724
Parameter of GRU is 900


*************Prposed GRU**********
MAE:  0.39260972
MSE:  0.31532136
RMSE:  0.5615348217680901
R2:  0.974119408419515
Parameter of GRU is 1000
*************Prposed GRU**********
MAE:  0.35135084
MSE:  0.27128273
RMSE:  0.5208480896448837
R2:  0.9777339632288545


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['SPIMACO'][0]}_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['SPIMACO'][0]}_predictions.csv")

### STC

In [ ]:
stc, stc_selected_ti = get_company_df(dataset,selected_companies_dic["STC"][0],selected_companies_dic["STC"][1])

/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-8-81f741971fba>:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

In [ ]:
train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(stc, stc_selected_ti)

Train shape: (4724, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (4714, 5, 34)
y shape: (4714, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)


In [ ]:
evaluation_df,predictions_df = train_evalute_models_for_company(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,selected_companies_dic["STC"][0])

Parameter of LSTM is 2


*************Prposed LSTM**********
MAE:  7.0416985
MSE:  85.57212
RMSE:  9.250520021409818
R2:  0.5893906484755898


Parameter of LSTM is 4
*************Prposed LSTM**********
MAE:  2.0412643
MSE:  8.378539
RMSE:  2.894570621938284
R2:  0.9597964035658381


Parameter of LSTM is 8
*************Prposed LSTM**********
MAE:  3.9520507
MSE:  40.430702
RMSE:  6.35851415107906
R2:  0.8059972173267274


Parameter of LSTM is 16
*************Prposed LSTM**********
MAE:  1.4850097
MSE:  4.709523
RMSE:  2.1701435899471653
R2:  0.9774018140332086


...

*************Prposed GRU**********
MAE:  1.2891605
MSE:  3.8793654
RMSE:  1.9696104803192305
R2:  0.9813852474921932
Parameter of GRU is 600


*************Prposed GRU**********
MAE:  1.2936366
MSE:  3.910942
RMSE:  1.9776101935509736
R2:  0.9812337326630362
Parameter of GRU is 700


*************Prposed GRU**********
MAE:  1.3955691
MSE:  4.2418847
RMSE:  2.0595836250088366
R2:  0.9796457275212253
Parameter of GRU is 800


*************Prposed GRU**********
MAE:  1.317744
MSE:  4.0421247
RMSE:  2.010503605624715
R2:  0.9806042602533227
Parameter of GRU is 900


*************Prposed GRU**********
MAE:  1.2994119
MSE:  3.9362748
RMSE:  1.9840047295613983
R2:  0.981112174631262
Parameter of GRU is 1000
*************Prposed GRU**********
MAE:  1.3783909
MSE:  4.1131535
RMSE:  2.028091087116553
R2:  0.980263440812637


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['STC'][0]}_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['STC'][0]}_predictions.csv")

### TASNEE

In [ ]:
tas, tas_selected_ti = get_company_df(dataset,selected_companies_dic["TASNEE"][0],selected_companies_dic["TASNEE"][1])

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-8-81f741971fba>:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  company['natr'] = ta.nat

In [ ]:
train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(tas, tas_selected_ti)

Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 34)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)


In [ ]:
evaluation_df,predictions_df = train_evalute_models_for_company(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,selected_companies_dic["TASNEE"][0])

Parameter of LSTM is 2


*************Prposed LSTM**********
MAE:  0.5609405
MSE:  0.60940176
RMSE:  0.7806418913211769
R2:  0.9159565745192361


Parameter of LSTM is 4
*************Prposed LSTM**********
MAE:  0.62163186
MSE:  0.787142
RMSE:  0.8872102224054298
R2:  0.8914441868812936


Parameter of LSTM is 8
*************Prposed LSTM**********
MAE:  0.38038364
MSE:  0.28991008
RMSE:  0.5384329838045827
R2:  0.9600180903960753


Parameter of LSTM is 16
*************Prposed LSTM**********
MAE:  0.4041326
MSE:  0.29597527
RMSE:  0.5440360906108302
R2:  0.959181644539367


...

*************Prposed GRU**********
MAE:  0.3555043
MSE:  0.25208712
RMSE:  0.5020827782761077
R2:  0.9652343139050348
Parameter of GRU is 600


*************Prposed GRU**********
MAE:  0.32365122
MSE:  0.2051255
RMSE:  0.4529078226211546
R2:  0.9717108568600482
Parameter of GRU is 700


*************Prposed GRU**********
MAE:  0.3138313
MSE:  0.19815949
RMSE:  0.4451510822803581
R2:  0.9726715516441851
Parameter of GRU is 800


*************Prposed GRU**********
MAE:  0.31223693
MSE:  0.19816013
RMSE:  0.4451518019791737
R2:  0.9726714681858177
Parameter of GRU is 900


*************Prposed GRU**********
MAE:  0.31966394
MSE:  0.2043722
RMSE:  0.4520754332958192
R2:  0.9718147551723137
Parameter of GRU is 1000
*************Prposed GRU**********
MAE:  0.31397536
MSE:  0.19838703
RMSE:  0.44540658592661625
R2:  0.9726401756097756


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['TASNEE'][0]}_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/results/{selected_companies_dic['TASNEE'][0]}_predictions.csv")